In [ ]:
!pip install google-cloud google-cloud-secret-manager

In [4]:
from google.cloud import secretmanager
from google.colab import auth


def get_secret(secret_name, proj='aigamer-382421'):
    auth.authenticate_user()
    client = secretmanager.SecretManagerServiceClient()
    response = client.access_secret_version(request={"name": f"projects/{proj}/secrets/{secret_name}/versions/latest"})
    return response.payload.data.decode('UTF-8')


def get_aws_creds():
    aws_key = get_secret('aws-key')
    aws_secret = get_secret('aws-secret')
    return aws_key, aws_secret


def get_github_creds(out_path='github-key'):
    key_str = get_secret('github-key')
    with open(out_path, 'w') as f:
        f.write(key_str)


In [15]:
!git clone https://github.com/yetanotheruseless/labs.git

Cloning into 'labs'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 73 (delta 21), reused 53 (delta 11), pack-reused 0
Unpacking objects: 100% (73/73), 25.43 KiB | 1.70 MiB/s, done.


In [16]:
!cd labs/src

In [19]:
!pwd

/content


In [32]:
aws_key, aws_secret = get_aws_creds()

In [20]:
import sys
sys.path.insert(0, '/content/labs/src')

In [26]:
!cd labs && pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/labs (from -r requirements.txt (line 2))
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 5.7 MB/s 

In [28]:
from data.make_dataset import *

In [38]:
books_prefix = 'projects/AI-RPG/interim/'
core_rulebook = books_prefix + "The_Strange_Consumer_PDF_Bookmarked_and_Linked_2015-06-23-text.tgz"
out_base = 'data'
!mkdir data


In [39]:
download_and_uncompress_tgz(aws_key, aws_secret, 'yetanotheruseless-data', core_rulebook, 'data/core')

Downloaded and uncompressed projects/AI-RPG/interim/The_Strange_Consumer_PDF_Bookmarked_and_Linked_2015-06-23-text.tgz from bucket yetanotheruseless-data to data/core


In [40]:
create_data_splits('data/core/Strange_Core_Rulebook', 'data/training')

In [41]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 27.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [42]:
from models.train_model import *


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), Po

In [43]:
model, tokenizer = get_model_and_tokenizer()

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /content/labs/src/models/train_model.py:31 in get_model_and_tokenizer                            │
│                                                                                                  │
│   28 def get_model_and_tokenizer(model_name="facebook/opt-6.7b", torch_dtype=torch.float32):     │
│   29 │   os.environ["CUDA_VISIBLE_DEVICES"] = "0"                                                │
│   30 │   device = torch.device("cuda" if torch.cuda.is_available() else "cpu")                   │
│ ❱ 31 │   model = AutoModelForCausalLM.from_pretrained(                                           │
│   32 │   │   model_name,                                                                         │
│   33 │   │   load_in_8bit=True,                                                                  │
│   34 │   │   device_map='auto',                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:1883 in to                 │
│                                                                                                  │
│   1880 │   def to(self, *args, **kwargs):                                                        │
│   1881 │   │   # Checks if the model has been loaded in 8-bit                                    │
│   1882 │   │   if getattr(self, "is_loaded_in_8bit", False):                                     │
│ ❱ 1883 │   │   │   raise ValueError(                                                             │
│   1884 │   │   │   │   "`.to` is not supported for `8-bit` models. Please use the model as it i  │
│   1885 │   │   │   │   " model has already been set to the correct devices and casted to the co  │
│   1886 │   │   │   )                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: `.to` is not supported for `8-bit` models. Please use the model as it is, since the model has already 
been set to the correct devices and casted to the correct `dtype`.

In [47]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM



def get_model_and_tokenizer(model_name="facebook/opt-6.7b", torch_dtype=torch.float32):
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_8bit=True,
        device_map='auto',
    )
    model = model.half() if torch_dtype == torch.float16 else model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    for param in model.parameters():
        param.requires_grad = False  # freeze the model - train adapters later
    if param.ndim == 1:
        # cast the small parameters (e.g. layernorm) to fp32 for stability
        param.data = param.data.to(torch.float32)
    model.gradient_checkpointing_enable()  # reduce number of stored activations
    model.enable_input_require_grads()
    model.lm_head = CastOutputToFloat(model.lm_head)
    config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, config)
    print_trainable_parameters(model)
    return model, tokenizer


model, tokenizer = get_model_and_tokenizer()

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 8388608 || all params: 6666862592 || trainable%: 0.12582542214183376


In [48]:
import transformers
from datasets import load_dataset
data = load_dataset("data/training")
data = data.map(lambda samples: tokenizer(samples['text']), batched=True)

trainer = transformers.Trainer(
    model=model, 
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        max_steps=200, 
        learning_rate=2e-4, 
        fp16=True,
        logging_steps=1, 
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Resolving data files:   0%|          | 0/334 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/12629 [00:00<?, ? examples/s]

...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,3.982800
2,5.837900
3,4.957000
4,4.784900
5,4.298300
6,5.026800
7,4.588200
8,5.038600
9,5.080400
10,4.293900


TrainOutput(global_step=200, training_loss=4.037242288589478, metrics={'train_runtime': 769.3841, 'train_samples_per_second': 4.159, 'train_steps_per_second': 0.26, 'total_flos': 6535454063984640.0, 'train_loss': 4.037242288589478, 'epoch': 0.25})

In [50]:
batch

{'input_ids': tensor([[    2,  9058,   383,    32, 32952,    35,  1437]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [58]:
batch = tokenizer("The difference between Ruk and Ardeyn is that:", return_tensors='pt')
with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, min_length=100, max_new_tokens=200, temperature=0.1)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1405: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")




 The difference between Ruk and Ardeyn is that:  Ruk is a                                                                                                                                                                                                    
